In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from pandas import Series, DataFrame
import pandas as pd

In [2]:
# driver = webdriver.Chrome('/Users/jungwoosong/Documents/python/chromedriver')
driver = webdriver.Firefox(executable_path='/Users/jungwoosong/Documents/python/geckodriver')

In [3]:
driver.set_window_position("0","0")
driver.set_window_size("1280","800")

# Categories

### Business & Finance

In [4]:
category = 'Business & Finance'
cur_category = category

### Total Scraping

In [5]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
reviews = list()
comments = list()
exitFlag = False
for i in range(2):
# while(exitFlag == False):
    time.sleep(2)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
#     for i in range(0,len(results)):
    for i in range(0,len(results)-8): #테스트용도로 페이지마다 8개씩만
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))
#     time.sleep(2)
    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
        if(soup.select("#dp-summary-see-all-reviews") != []): #review가 있을 때
            reviews_link = 'https://www.amazon.com'+soup.select("#dp-summary-see-all-reviews")[0]['href'] #review link가 없는 경우도 있다 # coin watch처럼
            driver.get(reviews_link)
            driver.find_element_by_css_selector(".reviewer-type-select").click() #파폭(맥북스크린)
            all_reviewers_flag = driver.find_element_by_css_selector("#reviewer-type-dropdown > option").text
            if(all_reviewers_flag == 'All reviewers'):
                print('yes, it is all reviewers')
                driver.find_element_by_css_selector("#reviewer-type-dropdown_0").click()
            else:
                print('wrong filter_not All reviewers')
                break
            while(True):
                time.sleep(2) # wait 2secs
                comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment
                for button in comments_subComment_toggle_list:
                    print(button)
                    button.location_once_scrolled_into_view
                    element = WebDriverWait(driver, 10).until(
                        EC.visibility_of(button)
                    )
                    element.click() #click all subcomments toggle
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
                    if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
                        break
                    review = {}
                    review['skill_title'] = cur_skill_Title
                    review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
                    review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
                    review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
                    review['rating'] = review_data.div.a['title'] 
                    review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
                    if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
                        review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
                    else:
                        review['vote'] = '0'
                    comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
                    if(comments_data.div is not None): #comment가 있다면
                        print("there are comments")        
                        for i, comment_data in enumerate(comments_data):
                            if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
                                comment = {}   
                                comment['review_title'] = review['title']
                                comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
                                comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
                                comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
                                comments.append(comment)
                    else:
                        print("no comments")
                    review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
                    review['timestamp'] = time.time()
                    reviews.append(review)
                if(soup.select("#cm_cr-pagination_bar > ul > li.a-last") != []):
                    if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
                        print('This is not a last page')
                        driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
                    else:
                        print('This is a last page')
                        print('Move to next Skill')
                        break
                else:
                    print('There are no other pages')
                    break
        else: #review가 없을 때
            print('No review link')        
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break

Rating exist
rating_raw = 3.5 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="1e25438c-c758-574f-b543-b2d063e078a5")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="b62ca5db-26f2-d044-af5a-9aea8b28e1e2")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="90157ed7-615b-c94a-bf8e-1eb59e62fed7")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="59bfe1ca-616d-3844-9ac0-e6c0604bba86")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="db5fd8ca-3b4f-d446-a27e-ce100910e0e7")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", elemen

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="bf8d3249-710c-7c47-8ce1-7089be72c470")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="3cc49136-7b17-f44f-845e-b4062818f918")>
no comments
no comments
no comments
there are comments
there are comments
there are comments
there are comments
there are comments
there are comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="0189b1d8-2c7e-3846-9bb1-8b247b6d1fa9")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="2b551396-342b-c347-9de5-6b47a3f8f1cb")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="68df1a78-60fc-4b41-b567-b29e7986811e")>
<selenium.webdriver.firefox.webelement.

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="ae68e5aa-e230-b14c-a300-529124d29f05")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="940e5fde-5820-1c48-b4a3-744debbdbc53")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="799ac7b7-a3b9-164a-91f0-584a6e5ebc48")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="e00bfaa2-9351-1b40-b496-423ead9fba48")>
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="aa8dd104-8737-d74e-9e8b-d4cbb52825d0")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="7428a3c9-390d-9d4d-b880-fa0c56923473")>
no comments
no comments
no comments
no comments
no comments
This is a last page
Move to next Skill
Rating exist
rating_raw = 5.0 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="b8a52210-7939-ef4c-a464-d14d0a31fd49")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="2d03e1f8-f091-4d43-991d-12f010716c75")>
no comments
no comments
There are no other pages
Rating exist
rating_raw = 4.0 out of 5 stars
invocation does not exist
invocation does not exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="079b2883-9bfb-6346-8974-cd6ff386e4ba"

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="39c3c59c-91fb-074d-9c78-f07422446bf6")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="a7598058-633d-6946-9661-99ceb41a84d4")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="e4bc04f0-7083-df49-9994-a21e79362cbf")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="a6e706d4-02b9-ef4f-a70c-2dbe4f14af32")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="5451d378-151f-3644-bf15-8f245f9048ec")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5fbc7c0-0cc8-3c44-adb4-e30b006ff6f1", element="eba44350-5816-9b4a-83c2-648b4d189623")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d5f

In [6]:
len(skills)

16

In [7]:
len(reviews)

223

In [8]:
len(comments)

56

In [12]:
skills_df = DataFrame(skills)

In [13]:
reviews_df = DataFrame(reviews)

In [14]:
comments_df = DataFrame(comments)

In [18]:
skills_df.to_csv('./skills_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [16]:
reviews_df.to_csv('./reviews_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [17]:
comments_df.to_csv('./comments_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')